This document will contain the template and codes for creating master data dictionary , carrying out basic and advanced
text analytics to prepare training set data be be feeeded into an machine learning supervised model to classify
product descriptions into multi level multi class classification/categories. 

Creating the Master Data Dictionary of Product Master Classes (Nouns)
=====================================


In [1]:
# Importing all necessary libraries
import time
start = time.time()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import nltk
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

In [2]:
# Import the dataset contaning the texts showing mapping between all possible  Noun,Modifier and Characteristic
raw_data_catelogue = pd.read_csv('Master_Dictionary.csv')
raw_data_catelogue.drop_duplicates(subset=None, inplace=True)
raw_data_catelogue = raw_data_catelogue.fillna('NA')

In [3]:
raw_data_catelogue.head(10)

,Noun,Modifier,Characteristic,TheValue
0,ABRASIVE,AAAA,NA,NA
1,ABRASIVE,BELT,BOND/BACKING,GLUE BOND
2,ABRASIVE,BELT,BOND/BACKING,RAYON BACKING
3,ABRASIVE,BELT,BOND/BACKING,RESIN BOND
4,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT CLOTH BACKING, CLOSE COATED"
5,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT COTTON BACKING, OPEN COATED"
6,ABRASIVE,BELT,BOND/BACKING,WATERPROOF
7,ABRASIVE,BELT,GRIT,80
8,ABRASIVE,BELT,GRIT,120
9,ABRASIVE,BELT,GRIT,180


In [4]:
import warnings
warnings.filterwarnings('ignore')

defDict = defaultdict(LabelEncoder)
data_dictionary_encoded = raw_data_catelogue.apply(lambda x: defDict[x.name].fit_transform(x))
orig_raw_data_catelogue = data_dictionary_encoded.apply(lambda x: defDict[x.name].inverse_transform(x))

In [5]:
orig_raw_data_catelogue.head(10)

,Noun,Modifier,Characteristic,TheValue
0,ABRASIVE,AAAA,NA,NA
1,ABRASIVE,BELT,BOND/BACKING,GLUE BOND
2,ABRASIVE,BELT,BOND/BACKING,RAYON BACKING
3,ABRASIVE,BELT,BOND/BACKING,RESIN BOND
4,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT CLOTH BACKING, CLOSE COATED"
5,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT COTTON BACKING, OPEN COATED"
6,ABRASIVE,BELT,BOND/BACKING,WATERPROOF
7,ABRASIVE,BELT,GRIT,80
8,ABRASIVE,BELT,GRIT,120
9,ABRASIVE,BELT,GRIT,180


In [6]:
data_dictionary_encoded.head()

,Noun,Modifier,Characteristic,TheValue
0,0,2,616,15012
1,0,54,88,13252
2,0,54,88,16157
3,0,54,88,16314
4,0,54,88,16322


In [7]:
data_dictionary_mapping = pd.concat([orig_raw_data_catelogue,data_dictionary_encoded], axis=1)
headers=['Noun','Modifier','Characteristic','TheValue','Noun_en','Modifier_en','Characteristic_en','TheValue_en']
data_dictionary_mapping.columns=headers

In [8]:
data_dictionary_mapping.head()

,Noun,Modifier,Characteristic,TheValue,Noun_en,Modifier_en,Characteristic_en,TheValue_en
0,ABRASIVE,AAAA,NA,NA,0,2,616,15012
1,ABRASIVE,BELT,BOND/BACKING,GLUE BOND,0,54,88,13252
2,ABRASIVE,BELT,BOND/BACKING,RAYON BACKING,0,54,88,16157
3,ABRASIVE,BELT,BOND/BACKING,RESIN BOND,0,54,88,16314
4,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT CLOTH BACKING, CLOSE COATED",0,54,88,16322


In [9]:
#Saving the dictionary to a file for reference
data_dictionary_mapping.to_csv('data_dictionary_mapping.csv', sep=',')

Basic feature extraction using text data
========================================

In [225]:
import re
manufacturer_ref_nr = pd.read_csv('Manufacture_Ref_No.csv')

def cleaning_input_text(input_text):
    text = re.sub("[,.]", " ", input_text)
    tokens = nltk.word_tokenize(text)
    all_words_bag_temp_1 = nltk.pos_tag(tokens)
    all_words=[]
    for j in range(len(all_words_bag_temp_1)):
        if(all_words_bag_temp_1[j][1] not in ['JJ','MD','PRP','CD','POS','IN'] and all_words_bag_temp_1[j][0].isalpha()):
            all_words.append(all_words_bag_temp_1[j][0])
    clean_text=' '.join(all_words)
    clean_text_1 = clean_text
    del clean_text
    return clean_text_1

def ngram_model(text,n_val,char_value):
    from nltk import ngrams
    sentence = text
    n = n_val
    ngrams = ngrams(sentence.split(), n)
    for grams in ngrams:
        str1 = " ".join(list(grams))
        return_val='NA'
        if (str1.upper() == char_value.upper()):
            return_val=char_value.upper()
            return (return_val)
            break
        else:
            return_val='NA'

def noun_identification(test_sample):
    from nltk import ngrams
    inp = cleaning_input_text(test_sample)
    word_tokens = nltk.word_tokenize(inp)
    noun = inp.split(' ', 1)[0].upper()
    if(noun in data_dictionary_mapping['Noun'].unique()):
        modifier_list = data_dictionary_mapping[data_dictionary_mapping['Noun'] == noun]['Modifier'].unique()
        for j in modifier_list:
            n_gram = len(j.split())
            modifier_val = ngram_model(inp,n_gram,j.upper())
            if (modifier_val != 'NA' and modifier_val is not None):
                dict_noun_modifier = {'noun':noun.upper(),'modifier':modifier_val.upper()}
                return dict_noun_modifier
        if (modifier_val == 'NA' or modifier_val is None):
            dict_noun_modifier = {'noun':noun.upper(),'modifier':'NO MODIFIER'}
            return dict_noun_modifier
    else:
        dict_noun_modifier = {'noun':'NO NOUN','modifier':'NO MODIFIER'}
        return dict_noun_modifier

def manufacturer(text):
    inp = text.replace(':',' : ').upper()
    manufacturer_names = manufacturer_ref_nr['Manufacturer'].str.upper().dropna()
    for i in manufacturer_names:
        if ((i in inp) and (':' in inp)):
            start_index = inp.index(i)
            manufacturer = inp[start_index:].split(':')[1].strip().rsplit(' ', 1)[0]
            return manufacturer
        elif ((i in inp) and (':' not in inp)):
            #inp_clean = re.sub("[,.]", " ", inp)
            list_of_words = inp.split()
            if (i in list_of_words):
                manufacturer = list_of_words[list_of_words.index(i) + 1]
                return manufacturer
        else:
            manufacturer='NA'
    return manufacturer

def ref_no(text):
    inp = text.replace(':',' : ').upper()
    ref_no_names = manufacturer_ref_nr['Ref_No'].str.upper().dropna()
    ref_no=''
    for i in ref_no_names:
        if ((i in inp) and (':' in inp)):
            start_index = inp.index(i)
            val = str(inp[start_index:].split(':')[1].strip().rsplit(' ', 1)[0])
            if (val not in ref_no):
                ref_no = str(ref_no) + val + str('\n')
        elif ((i in inp) and (':' not in inp)):
            list_of_words = inp.split()
            if (i in list_of_words):
                val1 = str(list_of_words[list_of_words.index(i) + 1])
                if (val1 not in ref_no):
                    ref_no = str(ref_no) + val1 + str('\n')
    if (len(ref_no)>0):
        return ref_no
    else:
        return 'NA'

In [226]:
k = ref_no("MOTOR,2.2 KW,400 V,KA57TDV100M4BMG2HF \
MANUFACTURER : HEXAWARE  MODEL_NUMBER_OR_TYPE_NUMBER: MNBVC \
Part Number: QWE \
EQPT_MNFR : ABC PHASE 3 SPEED  1400/47 VOLTAGE 400 SEW-EURODRIVE/KA57TDV100M4BMG2HF")
print(k)

MNBVC PART
QWE
ABC PHASE 3 SPEED  1400/47 VOLTAGE 400



In [231]:
def testing_module(choice,test_input):
    if (choice == '1'):
        df_raw_test = pd.read_csv("Test_Data.csv",encoding = "latin")
        df_raw_test.reset_index(drop=True, inplace=True)
        output_df = pd.DataFrame(columns=['Prod_ID','Material_Desc','Noun','Modifier'])
        counter1=df_raw_test.shape[0]
        
        for i in range(counter1):
            if(i%1000 == 0):
                print(i,'Inside 1 if loop')
            prod_id = int(1000 + i)
            noun_modifier_dict = noun_identification(df_raw_test['Long_Desc'][i].replace(',',' '))
            noun_value=noun_modifier_dict['noun']
            modifier_value = noun_modifier_dict['modifier']
            concat = noun_value + " | " + modifier_value
            output_df = output_df.append({'Prod_ID':prod_id,'Material_Desc':df_raw_test['Long_Desc'][i],'Noun':noun_value,'Modifier':modifier_value},ignore_index=True)

        out_df_final = pd.DataFrame()
        output_char_value_df_temp = pd.DataFrame()
        df1 = pd.DataFrame()
        counter = df_raw_test.shape[0]
        dummy_data = {'Dummy': [np.NaN]}
        output_char_value_df_dummy = pd.DataFrame(data=dummy_data)
        #characteristic_list = set(data_dictionary_mapping.iloc[:,2])
        for i in range(counter):
            if(i%1000 == 0):
                print(i,'Inside 2 if loop')
            noun = output_df['Noun'][i]
            modifier = output_df['Modifier'][i]
            char_list = data_dictionary_mapping[(data_dictionary_mapping['Noun'] == noun) & (data_dictionary_mapping['Modifier'] == modifier) ]
            char_values_all = char_list['Characteristic'].unique()
            output_char_value_df = pd.DataFrame(columns=char_values_all)
            counter1 = len(char_values_all)
            for j in range(counter1):
                char_value = char_values_all[j]
                char_flag=None
                word_list=char_value.split()
                search = word_list[0]
                prod_desc=output_df['Material_Desc'][i]
                clean_string = prod_desc.replace(':', ' ').replace('-', ' ')
                list_of_words = clean_string.split()
                if (search in list_of_words):
                    last_index =list_of_words.index(search) + len(word_list)
                    if (len(list_of_words) > last_index):
                        char_flag = list_of_words[list_of_words.index(search) + len(word_list)]
                        output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                if (char_flag is None):
                    output_char_value_df=output_char_value_df.append({char_value:'NA'},ignore_index=True)
            output_char_value_df = output_char_value_df.fillna(method='ffill')
            output_char_value_df = output_char_value_df.tail(1)

            if (output_char_value_df.empty):
                col = output_char_value_df_dummy.columns
                for k in range(output_char_value_df_dummy.shape[1]):
                    col_name = col[k]
                    output_char_value_df.loc[0,col_name] = np.NaN
            output_char_value_df_temp = output_char_value_df_temp.append(output_char_value_df)
            output_char_value_df=output_char_value_df.iloc[0:0]
        
        output_char_value_df_temp.reset_index(drop=True, inplace=True)
        output_df.reset_index(drop=True, inplace=True)
        final_output_df = pd.concat([output_df,output_char_value_df_temp],axis=1)
        new = pd.DataFrame()
        new = final_output_df.iloc[:,5:]
        df_temp1 = final_output_df.loc[:,'Prod_ID':'Modifier']
        df = pd.DataFrame()
        
        for i in range(new.shape[0]):
            if(i%1000 == 0):
                print(i,'Inside 3 if loop')       
            short_desc=''
            long_desc=''
            val=''
            for j in new.columns:
                val = final_output_df.loc[i,j]
                if (val != 'NA'):
                    if(not pd.isnull(val)):
                        short_desc = str(short_desc) + str(val) + ','
                        long_desc =  str(long_desc) + str(j) + ' : ' + str(val) + '\n'
            df = df.append({'short_description':short_desc,'long_description':long_desc},ignore_index=True)
        
        
        df_formatter=pd.concat([df_temp1,df],axis=1)
        modifier_dict = pd.read_csv('modifier_abbreviation_dict.csv')
        modifier_dict.reset_index(drop=True, inplace=True)
        for i in range(df_formatter.shape[0]):
            if(i%1000 == 0):
                print(i,'Inside 4 if loop')

            new_long_desc = "*** " + str(df_formatter.loc[i,'Noun']) + ' , ' + str(df_formatter.loc[i,'Modifier']) + " *** " + '\n\n' + str(df_formatter.loc[i,'long_description'])
            modi_search = df_formatter.loc[i,'Modifier']
            if(modi_search in list(modifier_dict['Modifier'])):
                short_modifier =modifier_dict[modifier_dict['Modifier'] == modi_search ]['Abbreviation'].values[0]
                new_short_desc =  str(df_formatter.loc[i,'Noun']) + ' , ' + str(short_modifier) + ': ' + str(df_formatter.loc[i,'short_description'])
                new_short_desc = new_short_desc[:-1]
            else:
                new_short_desc = str(df_formatter.loc[i,'Noun']) + ',' + str(df_formatter.loc[i,'Modifier']) + ':' + str(df_formatter.loc[i,'short_description'])
                new_short_desc = new_short_desc[:-1]
            df_formatter.loc[i,'long_description'] = new_long_desc
            df_formatter.loc[i,'short_description'] = new_short_desc
        
        df_exception = pd.DataFrame()
        df_manufacture = pd.DataFrame()
        df_ref_no = pd.DataFrame()
        #print(df_formatter.shape[0])
        for k in range(counter):
            if(k%1000 == 0):
                print(k,'Inside 5 if loop')
            #Manufacturer
            manufacturer_val = manufacturer(final_output_df.loc[k,'Material_Desc'])
            if (manufacturer_val != 'NA'):
                df_manufacture = df_manufacture.append({'Manufacturer': manufacturer_val},ignore_index=True)
            else:
                df_manufacture = df_manufacture.append({'Manufacturer': np.NaN},ignore_index=True)
            
            ref_no_val = ref_no(final_output_df.loc[k,'Material_Desc'])
            if (ref_no_val != 'NA'):
                df_ref_no = df_ref_no.append({'Ref_no': ref_no_val},ignore_index=True)
            else:
                df_ref_no = df_ref_no.append({'Ref_no': np.NaN},ignore_index=True)
            
            #Exception
            mat_desc=df_temp1.loc[k,'Material_Desc'].replace(',',' ')
            exception = [w.upper() for w in nltk.word_tokenize(mat_desc) if w not in nltk.word_tokenize(df_formatter.loc[k,'long_description'])]
            exception = ' '.join(exception)
            if (len(exception)>0):
                df_exception = df_exception.append({'exception': exception},ignore_index=True)
            else:
                df_exception = df_exception.append({'exception': 'No exception found'},ignore_index=True)
            
        output_dataframe=pd.DataFrame()
        new_output_df=pd.DataFrame()
        new_output_df=final_output_df.iloc[:,4:]
        output_dataframe = pd.concat([df_formatter,df_manufacture,df_ref_no,df_exception,new_output_df],axis=1)
        output_dataframe.to_csv('final_output_df_file_import.csv')
        display(output_dataframe)
        print('Please check you local file system for output generated : File ----> final_output_df_file_import.csv')
        return output_dataframe


In [233]:
# import warnings
warnings.filterwarnings('ignore')

import time
start = time.time()

choice = input("Enter choice(1/2):")
if choice == '1':
    print('Make sure you have feeded your input in the Test_Data.csv')
    test_input=''
    pred_output_df = pd.DataFrame()
    pred_output_df = testing_module(choice,test_input.upper())
if choice == '2':
    #test_input = input('Enter your test sample\n')
    test_input = "SPIRAL WOUND GASKET SIZE: 1""X 900/1500 LBS;WITH EXTERNAL GUIDE RING (STYLE-CG), WINDING MATERIAL:316L FILLER MATERIAL GRAPHITE AS PER ASME B 16.20 TO FIT FOR ANSI B 16.5 FLANGES"
    pred_output_df = testing_module(choice,test_input.upper())
    #display(pred_output_df)
end = time.time()
temp = end-start
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes
print(hours,':',minutes,':',seconds)

Enter choice(1/2):1
Make sure you have feeded your input in the Test_Data.csv
0 Inside 1 if loop
0 Inside 2 if loop
0 Inside 3 if loop
0 Inside 4 if loop
0 Inside 5 if loop


,Prod_ID,Material_Desc,Noun,Modifier,long_description,short_description,Manufacturer,Ref_no,exception,API NUMBER,...,SERVICE FACTOR,SHAFT SIZE,SHAPE,SIZE,SPECIFICATION,SPEED,STYLE,THICKNESS,TYPE,WINDING MATERIAL
0,1000,"MOTOR,ELECTRIC,15HP,190/380V,1475,50HZ\nINDUCT...",MOTOR,NO MODIFIER,"*** MOTOR , NO MODIFIER *** \n\nPOWER : 15\nSH...","MOTOR , NO MOD: 15,OPPOSITE,1475",HEXAWARE,HEX\nA/S2345\n,ELECTRIC 15HP 190/380V 50HZ INDUCTION FURNACE ...,NaN,...,NA,OPPOSITE,NaN,NaN,NaN,1475,NaN,NaN,NA,NaN
1,1001,"LSDP CONTROL\nSPIRAL WOUND GASKET 24"""""""" X 300...",NO NOUN,NO MODIFIER,"*** NO NOUN , NO MODIFIER *** \n\nMATERIAL : C...","NO NOUN , NO MOD: CERAMIC,3/4""""",NaN,NaN,LSDP CONTROL SPIRAL WOUND GASKET 24 X 300 # RF...,NaN,...,NaN,NaN,NaN,"3/4""""",NaN,NaN,NaN,NaN,NA,NaN
2,1002,"MOTOR,22 KW,400-690 KW,DFV180L4/BM/HR/TH\nNO G...",MOTOR,NO MODIFIER,"*** MOTOR , NO MODIFIER *** \n\nSPEED : 1465\n","MOTOR , NO MOD: 1465",NaN,NaN,22 KW 400-690 KW DFV180L4/BM/HR/TH GEAR BOX EQ...,NaN,...,NA,NA,NaN,NaN,NaN,1465,NaN,NaN,NA,NaN
3,1003,GASKET SPIRAL WOUND SIZE GRAPHITE FILLER MATER...,GASKET,SPIRAL WOUND,"*** GASKET , SPIRAL WOUND *** \n\nFILLER MATER...","GASKET , SPL WND: INNER,MONEL,STEEL,10"""""""",150...",NaN,NaN,LB 10-3/8 ID X 15 OD CG FURON RW LAMONS WR C N...,NaN,...,NaN,NaN,NaN,GRAPHITE,NaN,NaN,FLEXITALLIC,"1/16""""""""",NaN,HASTELLOY
4,1004,"MOTOR,7.5 KW,400/690 V D/Y,14.3/8.27A\nMOTOR F...",MOTOR,HYDRAULIC,"*** MOTOR , HYDRAULIC *** \n\nSPEED : 1455/1760\n","MOTOR , HYD: 1455/1760",NaN,NaN,7.5 KW 400/690 V D/Y 14.3/8.27A FOR PASTE COOL...,NaN,...,NaN,NaN,NaN,NaN,NaN,1455/1760,NaN,NaN,NA,NaN
5,1005,"LSDP CONTROL\nSPIRAL WOUND GASKET 8"""""""" X 150#...",NO NOUN,NO MODIFIER,"*** NO NOUN , NO MODIFIER *** \n\nMATERIAL : G...","NO NOUN , NO MOD: GRAPHITE",NaN,NaN,LSDP CONTROL SPIRAL WOUND GASKET 8 '' '' '' ''...,NaN,...,NaN,NaN,NaN,NA,NaN,NaN,NaN,NaN,NA,NaN
6,1006,"MOTOR,2.2 KW,400 V,KA57TDV100M4BMG2HF\nMANUFAC...",MOTOR,NO MODIFIER,"*** MOTOR , NO MODIFIER *** \n\nSPEED : 1400/47\n","MOTOR , NO MOD: 1400/47",INFO,NaN,2.2 KW 400 V KA57TDV100M4BMG2HF MANUFACTURER I...,NaN,...,NA,NA,NaN,NaN,NaN,1400/47,NaN,NaN,NA,NaN
7,1007,"GASKET,SPIRAL WOUND,RWI-RJ,2""\nFILLER MATERIAL...",GASKET,RING JOINT,"*** GASKET , RING JOINT *** \n\nMATERIAL : PTF...","GASKET , RJ: PTFE,ASME",NaN,NaN,SPIRAL WOUND RWI-RJ 2 '' FILLER FOR REPLACEMEN...,NA,...,NaN,NaN,NA,NaN,ASME,NaN,NaN,NaN,NaN,NaN
8,1008,"MOTOR,ELECTRIC,0.49KW,VEP15400800\nMOTOR_ELECT...",MOTOR,NO MODIFIER,"*** MOTOR , NO MODIFIER *** \n\nSHAFT SIZE : D...","MOTOR , NO MOD: DIRECT,1500RPM",NaN,NaN,ELECTRIC 0.49KW VEP15400800 MOTOR_ELECTRICAL V...,NaN,...,NA,DIRECT,NaN,NaN,NaN,1500RPM,NaN,NaN,NA,NaN
9,1009,"GASKET , SPIRAL WOUND FILLER MATERIAL GRAPHI...",GASKET,SPIRAL WOUND,"*** GASKET , SPIRAL WOUND *** \n\nFILLER MATER...","GASKET , SPL WND: GRAPHITE,3/4,300,7,FLEXITALL...",3/4,"ABCD\nPRESSURE RATING 300 LB SIZE 7-1/2"""" ID X...",SN QWEB P/N ABCD LB 7-1/2 ID X 8-1/2 OD CG 600...,NaN,...,NaN,NaN,NaN,7,NaN,NaN,FLEXITALLIC,"1/16""""",NaN,INCONEL


Please check you local file system for output generated : File ----> final_output_df_file_import.csv
0.0 : 0.0 : 1.8080756664276123


In [ ]:
pred_output_df = pred_output_df.sort_values(['Noun','Modifier'], ascending=[True,True])
n_noun_modifier= pd.DataFrame()
n_noun_modifier = pred_output_df.loc[:,'Noun':'Modifier'].drop_duplicates()
n_noun_modifier.reset_index(drop=True, inplace=True)
writer = pd.ExcelWriter('final_output_df_file_import_excel.xlsx', engine='xlsxwriter')

for i in range(n_noun_modifier.shape[0]):
    noun=n_noun_modifier.loc[i,'Noun'].replace('/','_')
    modifier=n_noun_modifier.loc[i,'Modifier'].replace('/','_')
    df_name = str(noun) + "_" + str(modifier)
    name = n_noun_modifier.loc[i,'Noun'].replace(' ','_').replace('/','_') + "_" + n_noun_modifier.loc[i,'Modifier'].replace(' ','_').replace('/','_')
    df_taxonomy = pd.DataFrame(index=None)
    df_taxonomy = pred_output_df[pred_output_df['Noun'] == noun]
    df_taxonomy = df_taxonomy[df_taxonomy['Modifier'] == modifier]
    df_taxonomy.dropna(axis='columns',inplace=True,how='all')
    df_taxonomy = df_taxonomy.replace('NA','')
    df_taxonomy.reset_index(drop=True, inplace=True)
    df_taxonomy.columns = map(str.upper, df_taxonomy.columns)
    workbook  = writer.book
    df_taxonomy.to_excel(writer,sheet_name=name)
    worksheet = writer.sheets[name]
    start_row = 1
    start_col = 8
    end_row = df_taxonomy.shape[0]
    end_col = df_taxonomy.shape[1]
    format0 = workbook.add_format({'bg_color': '#FFA07A'})
    format1 = workbook.add_format({'bg_color': '#90EE90'})
    format2 = workbook.add_format({'font_color': '#FFFFFF','bg_color': '#0000FF'})
    cell_format = workbook.add_format()
    cell_format.set_align('left')
    cell_format.set_align('top')
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'type':'blanks', 'format': format0})
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'type':'no_blanks', 'format': format1})
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'format': cell_format})
    worksheet.conditional_format(0, 0, 0, end_col, {'type':'no_blanks', 'format': format2})
writer.save()